In [ ]:
!pip3 install git+https://github.com/holounic/Bayesian-Optimization.git
!pip3 install cma

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/holounic/Bayesian-Optimization.git to /tmp/pip-req-build-e16ihy_g
  Running command git clone -q https://github.com/holounic/Bayesian-Optimization.git /tmp/pip-req-build-e16ihy_g
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Cloning https://github.com/holounic/pyDOE to /tmp/pip-install-p6cmwr5o/pydoe_3be7ad82adf7447eac811428166a0418
  Running command git clone -q https://github.com/holounic/pyDOE /tmp/pip-install-p6cmwr5o/pydoe_3be7ad82adf7447eac811428166a0418
     |████████████████████████████████| 316 kB 6.5 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
  Created wheel for bayes-optim: filename=bayes_optim-0.2.7-py3-none-any.whl size=145056 sha256=49759edd1499038ac44b5e0442db6279c180397a3c591059b3d5f57be6161848
  Stored in directory: /tmp/pip-ephem-wheel-cache-u

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

import cma

from pyDOE import lhs

from bayes_optim import BO
from bayes_optim.search_space import RealSpace
from bayes_optim.surrogate import GaussianProcess, trend

from benchmark.bbobbenchmarks import F12

from tqdm.notebook import trange

# Constants

In [ ]:
np.random.seed(1111)

dim = 2
lb, ub = -5, 5

mean = trend.constant_trend(dim, beta=None)
thetaL = 1e-10 * (ub - lb) * np.ones(dim)
thetaU = 10 * (ub - lb) * np.ones(dim)
theta0 = np.random.rand(dim) * (thetaU - thetaL) + thetaL

max_FEs = 2**5
verbose = False

doe_budget = 0.2
doe_size = int(doe_budget * max_FEs)

n_samples = 30000

max_cma_iter = 20

f = F12

test_seeds = list(range(50))

space = RealSpace([lb, ub]) * dim

pop_size = int(4 + 3 * np.log(doe_size * dim) // 1)

n_tests = 3

f_name = 'f12'

# Make optimizer function

In [ ]:
def make_optimizer(obj_func, doe):
    model = GaussianProcess(
        mean=mean,
        corr="squared_exponential",
        theta0=theta0,
        thetaL=thetaL,
        thetaU=thetaU,
        nugget=0,
        noise_estim=False,
        optimizer="BFGS",
        wait_iter=3,
        random_start=dim,
        likelihood="concentrated",
        eval_budget=100 * dim
    )
    return BO(
        search_space=space,
        obj_fun=obj_func,
        model=model,
        DoE_size=len(doe),
        acquisition_fun="EI",
        initial_points=doe,
        max_FEs=max_FEs,
        verbose=verbose,
        n_point=1
    )

# Experiment code

In [ ]:
def regret(obj_func, doe):
  bo = make_optimizer(obj_func, doe)
  xopt, yopt, _ = bo.run()
  return yopt - obj_func.getfopt()

def fitness_function(doe_size, n_individuals, func_seeds):
  i = 0
  def func(X):
    nonlocal i
    X = X.clip(lb, ub, out=X)
    doe = np.split(X, doe_size)
    obj_func = None
    k = i // n_individuals
    func_seed_set = func_seeds[k]
    i += 1
    r = 0
    print(func_seed_set)
    for func_seed in func_seed_set:
      r += regret(f(func_seed), doe)
    return r / len(func_seed_set)
  return func

def cma_es_experiment(doe_size, func_seeds, individual0, sigma0 = ub / 3):
  # run cma-es 
  doe_flattened, es = cma.fmin2(fitness_function(doe_size, pop_size, func_seeds), individual0, sigma0, {'maxiter': max_cma_iter, 'popsize': pop_size})
  
  # split a vector into set of points
  doe = np.split(doe_flattened, doe_size)

  return doe, es

# Testing code

In [ ]:
def additional_metrics(history):
  for h in history:
    func = f(h['f_s'])
    y = func(h['xopt'])
    real_x = func._getxopt()
    d = np.linalg.norm(h['doe'] - real_x, axis=1)
    h['min_dist_to_opt'] = d.min()
    h['max_dist_to_opt'] = d.max()
    h['real_xopt'] = real_x
    h['real_yopt'] = func.getfopt()
    h['bo_history'] = h['bo'].history
  return history

In [ ]:
def compute_all(func_seed, doe, n=3, n_points=1000):
  regrets, history, precisions = [], [], []
  for _ in range(n):
    obj_func = f(func_seed)
    bo = make_optimizer(obj_func, doe)
    xopt, yopt, _ = bo.run()
    regret = yopt - obj_func.getfopt()
    points = lhs(dim, n_points) * 10 - 5
    predicted_y = np.concatenate(bo.model.predict(points))
    real_y = obj_func(points)
    precision = np.mean((predicted_y - real_y)**2)
    precisions.append(precision)
    regrets.append(regret)
    history.append({'xopt': xopt, 'yopt': yopt, 'regret': regret, 'doe': doe, 'precision': precision, 'bo': bo, 'f_s': func_seed})
  return np.concatenate(regrets).mean(), np.mean(precisions), history

def test(doe, n=10):
  precisions, regrets, history = [], [], []
  for seed in trange(n):
    doe = doe
    r, p, h = compute_all(seed, doe, n=1, n_points=1000)
    precisions.append(p)
    regrets.append(r)
    history.append(h[0])
  return precisions, regrets, history

# Experiments

In [ ]:
individual0 = np.concatenate(lhs(2, doe_size) * 10 - 5)
individual0

array([ 3.82993797,  3.69797044, -3.58048274, -1.6711218 , -3.17007254,
       -0.99634532,  1.36891696,  0.51264257, -0.37135416,  1.88584949,
        2.78810256, -3.71311058])

In [ ]:
func_seeds = np.random.randint(0, 2**31, (max_cma_iter, n_tests)).tolist()
func_seeds.append(test_seeds)
func_seeds

[[9520565, 1264779723, 1077661487],
 [1577674721, 2113248958, 1270514153],
 [1999468482, 1791778377, 1561099687],
 [1164463400, 1739521995, 1120363494],
 [1199989895, 560111241, 1551970125],
 [340729745, 642745294, 562030683],
 [2081186164, 1289850646, 1622618882],
 [1384188607, 294536547, 692011541],
 [1002358442, 1353744663, 874605029],
 [109643978, 2146007785, 626418311],
 [721083299, 1076514125, 708533637],
 [240635361, 1585093152, 1607201968],
 [1784511006, 1519280398, 312385899],
 [1307519856, 920991999, 296144916],
 [2030458525, 735838891, 1491781517],
 [546898548, 1033767498, 1955096374],
 [1544998933, 542639558, 1905787870],
 [38628317, 1971868936, 1169363011],
 [1339508160, 997624450, 865132729],
 [1976130905, 1882654615, 1808760709],
 [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  4

In [ ]:
doe, es = cma_es_experiment(doe_size, func_seeds, individual0, sigma0 = ub / 3)

(5_w,11)-aCMA-ES (mu_w=3.4,w_1=42%) in dimension 12 (seed=690313, Wed Nov 16 16:23:15 2022)
[9520565, 1264779723, 1077661487]
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
[9520565, 1264779723, 1077661487]
[9520565, 1264779723, 1077661487]
Invalid likelihood value. Increasing nugget...
[9520565, 1264779723, 1077661487]
[9520565, 1264779723, 1077661487]
Invalid likelihood value. Increasing nugget...
[9520565, 1264779723, 1077661487]
Invalid likelihood value. Increasing nugget...
[9520565, 1264779723, 1077661487]
Invalid likelihood value. Increasing nugget...
[9520565, 1264779723, 1077661487]
[9520565, 1264779723, 1077661487]
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
[9520565, 1264779723, 1077661487]
[9520565, 1264779723, 1077661487]
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     11 1.839023155760945e+02 1.0e+00 1.51e+00  1e+00  2e+00 1:37.8
[1577674721, 

In [ ]:
doe

[array([2.17054301, 3.42772763]),
 array([-1.50978724, -1.77629882]),
 array([-2.11762885, -1.05516624]),
 array([-0.18978119,  4.14286828]),
 array([-1.79572371,  3.07236783]),
 array([ 4.29727019, -2.76536903])]

In [ ]:
precision, regrets, history = test(doe, n=50)

  0%|          | 0/50 [00:00<?, ?it/s]

Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likel

In [ ]:
np.mean(precision)

3.1987116465800954e+19

In [ ]:
pd.DataFrame(additional_metrics(history)).drop('bo', axis=1).to_csv(f'{f_name}_cma_es_doe.csv')

In [ ]:
doe_lhs = np.split(individual0, doe_size)
doe_lhs

[array([3.82993797, 3.69797044]),
 array([-3.58048274, -1.6711218 ]),
 array([-3.17007254, -0.99634532]),
 array([1.36891696, 0.51264257]),
 array([-0.37135416,  1.88584949]),
 array([ 2.78810256, -3.71311058])]

In [ ]:
lhs_precision, lhs_regrets, lhs_history = test(doe_lhs, n=50)

  0%|          | 0/50 [00:00<?, ?it/s]

Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likel

In [ ]:
np.mean(lhs_precision)

3.0113224404093596e+19

In [ ]:
pd.DataFrame(additional_metrics(lhs_history)).drop('bo', axis=1).to_csv(f'{f_name}_lhs_initial_doe.csv')